In [ ]:
!pip install -q xgboost==0.90

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
%store -r spark_processing_job_s3_output_prefix

In [ ]:
print('Previous Spark Processing Job Name: {}'.format(spark_processing_job_s3_output_prefix))

# Specify the S3 Location of the Features

In [ ]:
prefix_train = '{}/output/tfidf-train'.format(spark_processing_job_s3_output_prefix)
prefix_validation = '{}/output/tfidf-validation'.format(spark_processing_job_s3_output_prefix)
prefix_test = '{}/output/tfidf-test'.format(spark_processing_job_s3_output_prefix)

tfidf_train_path = './{}'.format(prefix_train)
tfidf_validation_path = './{}'.format(prefix_validation)
tfidf_test_path = './{}'.format(prefix_test)

tfidf_train_s3_uri = 's3://{}/{}'.format(bucket, prefix_train)
tfidf_validation_s3_uri = 's3://{}/{}'.format(bucket, prefix_validation)
tfidf_test_s3_uri = 's3://{}/{}'.format(bucket, prefix_test)

print(tfidf_train_s3_uri)
print(tfidf_validation_s3_uri)
print(tfidf_test_s3_uri)

In [ ]:
s3_input_train_data = sagemaker.s3_input(s3_data=tfidf_train_s3_uri, content_type='text/csv')
s3_input_validation_data = sagemaker.s3_input(s3_data=tfidf_validation_s3_uri, content_type='text/csv')
s3_input_test_data = sagemaker.s3_input(s3_data=tfidf_test_s3_uri, content_type='text/csv')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri 
import json

# get the URI for new container
builtin_container_uri = get_image_uri(region_name=region,                                
                                      repo_name='xgboost', 
                                      repo_version='0.90-2')
print(builtin_container_uri)

model_output_path = 's3://{}/models/built-in/training-runs'.format(bucket)
print(model_output_path)

xgb_estimator = sagemaker.estimator.Estimator(image_name=builtin_container_uri, 
                                              role=role, 
                                              train_instance_count=3,
                                              train_instance_type='ml.c5.4xlarge',
                                              output_path=model_output_path, 
                                              sagemaker_session=sess,
                                              # enable_cloudwatch_metrics=True
                                             )

xgb_estimator.set_hyperparameters(objective='binary:logistic',
                                  num_round=1,
                                  max_depth=5)


In [ ]:
xgb_estimator.fit({'train': s3_input_train_data,
                   'validation': s3_input_validation_data
                  }
                  #, wait=False
                 )

In [ ]:
training_job_name = xgb_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

In [ ]:
# TODO:  This isn't working at the moment
#from sagemaker.xgboost import XGBoost

#xgb_estimator = XGBoost.attach(training_job_name=training_job_name)

# Load the Model

In [ ]:
# download the model artifact from AWS S3
!aws s3 cp $model_output_path/$training_job_name/output/model.tar.gz ./models/built-in/

In [ ]:
import tarfile
import pickle as pkl

tar = tarfile.open('./models/built-in/model.tar.gz')
tar.extractall(path='./models/built-in/')
tar.close()

In [ ]:
!ls -al ./models/built-in/

In [ ]:
import pickle as pkl
import os

model_dir = './models/built-in'
model_path = os.path.join(model_dir, 'xgboost-model')

xgb_estimator_restored = pkl.load(open(model_path, 'rb'))

type(xgb_estimator_restored)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

import xgboost

fig, ax = plt.subplots(figsize=(12,12))
xgboost.plot_importance(xgb_estimator_restored, 
                        importance_type='gain', 
                        max_num_features=30, 
                        height=0.8, 
                        ax=ax, 
                        show_values = True)
plt.title('Feature Importance')
plt.show()

#  Calculate Test Metrics

## Download Test Dataset and Load into Memory

In [ ]:
!aws s3 ls --recursive $balanced_tfidf_without_header_test_s3_uri

In [ ]:
!aws s3 cp --recursive $balanced_tfidf_without_header_test_s3_uri $balanced_tfidf_without_header_test_path

In [ ]:
import glob
import pandas as pd

def load_dataset(path, sep, header):
    data = pd.concat([pd.read_csv(f, sep=sep, header=header) for f in glob.glob('{}/*.csv'.format(path))], ignore_index = True)

    labels = data.iloc[:,0]
    features = data.drop(data.columns[0], axis=1)
    
    if header==None:
        # Adjust the column names after dropped the 0th column above
        # New column names are 0 (inclusive) to len(features.columns) (exclusive)
        new_column_names = list(range(0, len(features.columns)))
        features.columns = new_column_names

    return features, labels

In [ ]:
X_test, y_test = load_dataset(path=balanced_tfidf_without_header_test_path, sep=',', header=None)
X_test.shape

In [ ]:
X_test.head(5)

In [ ]:
from xgboost import DMatrix

X_test.columns = ['f{}'.format(i) for i in range(0, 300)]

# Must convert pandas dataframe to XGBoost DMatrix before predicting
preds_test = xgb_estimator_restored.predict(DMatrix(X_test))
preds_test.shape

In [ ]:
preds_test

### Convert probability values into classification (0 or 1) using threshold 0.5

In [ ]:
import numpy as np
preds_test_0_or_1 = np.where(preds_test > 0.5, 1, 0)
preds_test_0_or_1.shape

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix

print('Test Accuracy: ', accuracy_score(y_test, preds_test_0_or_1))
print('Test Precision: ', precision_score(y_test, preds_test_0_or_1, average=None))

In [ ]:
print(classification_report(y_test, preds_test_0_or_1))

In [ ]:
df_cm_test = confusion_matrix(y_test, preds_test_0_or_1)
df_cm_test

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

def plot_conf_mat(cm, classes, title, cmap = plt.cm.Greens):
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
        horizontalalignment="center",
        color="black" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

# Plot non-normalized confusion matrix
plt.figure()
fig, ax = plt.subplots(figsize=(10,5))
plot_conf_mat(df_cm_test, classes=['Not Positive Sentiment', 'Positive Sentiment'], 
                          title='Confusion matrix')
plt.show()

In [ ]:
from sklearn import metrics

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

auc = round(metrics.roc_auc_score(y_test, preds_test_0_or_1), 4)
print('AUC is ' + repr(auc))

fpr, tpr, _ = metrics.roc_curve(y_test, preds_test_0_or_1)

plt.title('ROC Curve')
plt.plot(fpr, tpr, 'b',
label='AUC = %0.2f'% auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()